<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_Baraff_Witkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meshzoo

In [ ]:
from collections import namedtuple
import numpy as np
import meshzoo
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
vertex_positions_uv, triangle_indices = meshzoo.rectangle_tri(
    (0.0, 0.0),
    (1.0, 1.0),
    n=3,
    variant="zigzag",  # or "up", "down", "center"
)

print(vertex_positions_uv.shape)
print(triangle_indices.shape)

In [ ]:
x, y = np.transpose(vertex_positions_uv)
z = np.zeros_like(x)

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')

ax.plot_trisurf(x, y, z, triangles=triangle_indices, color='deepskyblue')
ax.scatter(x, y, z, c='deeppink', s=20, depthshade=False)
for i, (xi, yi, zi) in enumerate(zip(x, y, z)):
    ax.text(xi, yi, zi, str(i), fontsize='large', color='black', zorder=10)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

In [ ]:
def triangle_area(triangle_vertices):
    v0, v1, v2 = triangle_vertices
    return np.linalg.norm(np.cross(v1 - v0, v2 - v0)) / 2.0

def calculate_area(triangles_indices, vertex_positions):
    area = 0.0
    for triangle in triangle_indices:
        triangle_vertices = vertex_positions[triangle]
        area += triangle_area(triangle_vertices)
    return area

def calculate_vertex_masses(vertex_positions_uv, triangle_indices, density=0.2):
    vertex_masses = np.zeros(vertex_positions_uv.shape[0])

    for i,j,k in triangle_indices:
        area = triangle_area(vertex_positions_uv[[i, j, k]])
        triangle_mass = density * area
        vertex_masses[i] += 1/3 * triangle_mass 
        vertex_masses[j] += 1/3 * triangle_mass 
        vertex_masses[k] += 1/3 * triangle_mass

    return vertex_masses

def calculate_uv_stuff(triangle_vertex_positions_uv):
    uv_i, uv_j, uv_k = triangle_vertex_positions_uv
    ui, vi = uv_i
    uj, vj = uv_j
    uk, vk = uv_k

    delta_u1 = uj - ui
    delta_u2 = uk - ui
    delta_v1 = vj - vi
    delta_v2 = vk - vi

    delta_u_matrix = np.array([(delta_u1, delta_u2),
                               (delta_v1, delta_v2)])
    
    inverted_delta_u_matrix = np.linalg.inv(delta_u_matrix)

    dw_denominator = (delta_u1 * delta_v2 - delta_u2 * delta_v1)

    Xi_factor = (delta_v1 - delta_v2) / dw_denominator
    Xj_factor = delta_v2 / dw_denominator
    Xk_factor = -delta_v1 / dw_denominator

    return np.array([Xi_factor, Xj_factor, Xk_factor]), inverted_delta_u_matrix

def precompute_triangles_uv_stuff(vertex_positions_uv, triangle_indices):
    triangle_X_factors = []
    inverted_delta_u_matrices = []

    for triangle in triangle_indices:
        X_factors, inverted_delta_u_matrix = calculate_uv_stuff(vertex_positions_uv[triangle])
        triangle_X_factors.append(X_factors)
        inverted_delta_u_matrices.append(inverted_delta_u_matrix)
    
    return np.array(triangle_X_factors), np.array(inverted_delta_u_matrices)

In [ ]:
# Required attributes for a Baraff-Witkin style cloth simulator
class ClothSimulationMesh:
    # TODO rewrite cloth class with __init__():

    # State 
    vertex_positions = None              # float3
    vertex_velocities = None             # float3
    # Configuration / Parameters
    triangle_indices = None
    vertex_positions_uv = None           # float2
    vertex_masses = None                 # float
    triangle_stretch_stiffness_u = None  # float
    triangle_stretch_stiffness_v = None  # float
    pinned_vertices = None               # boolean
    # could also be saved as a sparse attribute

    # Simulation (intermediate/cache) storage
    vertex_forces = None
    triangle_areas_uv = None
    triangle_X_factors = None
    triangle_inverted_delta_u_matrices = None    # (2, 2) matrix on triangle domain 


n_vertices = vertex_positions_uv.shape[0]
n_triangles = triangle_indices.shape[0]

cloth = ClothSimulationMesh()
cloth.vertex_positions = np.transpose([x, y, z])
cloth.vertex_velocities = np.zeros_like(cloth.vertex_positions)

cloth.triangle_indices = triangle_indices
cloth.vertex_positions_uv = vertex_positions_uv
cloth.vertex_masses = calculate_vertex_masses(vertex_positions_uv, triangle_indices)
cloth.triangle_stretch_stiffness_u = np.ones(n_triangles)
cloth.triangle_stretch_stiffness_v = np.ones(n_triangles)
cloth.pinned_vertices = np.zeros(n_vertices)

cloth.vertex_forces = np.zeros_like(cloth.vertex_velocities)
cloth.triangle_areas_uv = np.array([triangle_area(vertex_positions_uv[tri]) for tri in triangle_indices])

triangle_X_factors, inverted_delta_u_matrices = precompute_triangles_uv_stuff(vertex_positions_uv, triangle_indices)
cloth.triangle_X_factors = triangle_X_factors
cloth.triangle_inverted_delta_u_matrices = inverted_delta_u_matrices

In [ ]:
for triangle_index, triangle_vertex_indicies in enumerate(triangle_indices):
    # given current vertex positions
    # find current stretch in u and v directions (w_u and w_v)
    # using the known rest state uv positions
    
    i, j, k = triangle_vertex_indicies

    # to avoid confusio between the x coordinate and the X position vector,
    # I'm going to capitalize the X when I mean the vector 
    # (position.x, position.y, position.z)
    # in the paper this is x printed in bold
    Xi, Xj, Xk = cloth.vertex_positions[triangle_vertex_indicies]

    area_uv = cloth.triangle_areas_uv[triangle_index]

    delta_X1 = Xj - Xi
    delta_X2 = Xk - Xi

    uv_i, uv_j, uv_k = vertex_positions_uv[triangle_vertex_indicies]
    ui, vi = uv_i
    uj, vj = uv_j
    uk, vk = uv_k

    delta_u1 = uj - ui
    delta_u2 = uk - ui
    delta_v1 = vj - vi
    delta_v2 = vk - vi

    delta_u_matrix = np.array([(delta_u1, delta_u2),
                               (delta_v1, delta_v2)])
        
    delta_X_matrix = np.column_stack((delta_X1, delta_X2))

    # Equation (9) in Baraff-Witkin.
    w_uv = delta_X_matrix @ np.linalg.inv(delta_u_matrix)
    wu, wv = np.hsplit(w_uv, 2)     # wu = "world u"
    norm_wu = np.linalg.norm(wu)
    norm_wv = np.linalg.norm(wv)

    # with the above we can easily find C(X), now we need to go further an find
    # dC(X)/dX
    # on a per-triangle basis, this is a vector with 9 elements.
    # the first 3 elements correspond to the first vertex i, the next 3 to j and the last 3 to k
    # so dC(Xi)/dXi is a 3 component vector, however, because the (x,y,z) 
    # components are treatly identically, we only beed to find the expression for
    # one of them and then we can substitute x for y or z to get the others. 

    # find dCu(Xi.x)/d(Xi.x)

    # Equation (10) in Baraff-Witkin.
    Cu = area_uv * (norm_wu - 1.0)

    dw_denominator = (delta_u1 * delta_v2 - delta_u2 * delta_v1)

    Xi_factor = (delta_v1 - delta_v2) / dw_denominator
    Xj_factor = delta_v2 / dw_denominator
    Xk_factor = -delta_v1 / dw_denominator

    wu_normalized = wu / norm_wu

    # It would have been nice if these derivates where give in the original paper,
    # sadly they weren't so I had to derive them myself by hand. I also found
    # this blog online with the derivation, however I was confused a bit about the 
    # notation at first, so I'll make my own blog post. 
    # blog: http://davidpritchard.org/freecloth/docs/report-single/

    dCu_Xi = area_uv * Xi_factor * wu_normalized  # it would be slightly more effecient to multiply all the scalars first and only then to the vector multiply
    dCu_Xj = area_uv * Xj_factor * wu_normalized
    dCu_Xk = area_uv * Xk_factor * wu_normalized

    u_stiffness = v_stiffness = 1.0
    # Equation (7) in Baraff-Witkin.
    force_i = u_stiffness * Cu * dCu_Xi
    force_j = u_stiffness * Cu * dCu_Xi
    force_k = u_stiffness * Cu * dCu_Xi

    cloth.vertex_forces[i] += force_i.reshape((3,)) # Reshape from (3, 1) to (3,)
    cloth.vertex_forces[j] += force_j.reshape((3,))
    cloth.vertex_forces[k] += force_k.reshape((3,))

    # todo think about a lockfree way to fill force vector
    # force vector has 1 element per vertex, but most vertices are contained in multiple triangles
    # so if we compute the triangle forces in parallel, we need a lock when writing to the vertex position

    # maybe: one pass over triangles -> local storage per triangle

    # solution parallel pass over triangles -> store calculated forces locally
    # then sequential pass over triangles that copies triangle forces into forces vector

    # important abstraction later: function that fill in forces vector
    # almost all sims work with forces applied to particles
    # in BW98 via triangle based stretch condition (in Blender if mesh is passed with quads, just use quad face attribute on looptris)
    # in MSD sims via edge based hooke's law

    # also: function that fills force jacobians matrix

In [ ]:
# def simulate(cloth, timesteps, dt):
#     history = [cloth.vertex_positions]
#     for _ in range(timesteps):
#         step(mesh, dt)
#         history.append(cloth.vertex_positions)

#     return history

# history = simulate(cloth, timesteps, dt)